In [1]:
# Add paths

import os
import sys
from dotenv import load_dotenv

load_dotenv(override=True)

paths = [
    os.path.abspath(os.path.join(os.getcwd(), "..")),
    os.path.abspath(os.path.join(os.getcwd(), "..", "agent")),
]

for dir_path in paths:
    if dir_path not in sys.path:
        sys.path.insert(0, dir_path)

In [2]:
import pandas as pd
from datetime import datetime
from langgraph.types import Command
from langchain_core.runnables import RunnableConfig
from agent.state import Data, DataType
from agent.entry_graph import g

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

config = RunnableConfig(
    {
        "configurable": {
            "thread_id": current_time,
        },
        "recursion_limit": 1000,
    }
)

user_input = {
    "question": "Can you find underrated 2-bedroom apartment units?",
    "city": "Melbourne",
    "purpose": "investment",
    "rooms": 2,
    "type": "house",
    "budget": "600k-900k",
    "top_n": 10,
    "investment_timeline_in_years": 10,
    "method": "Create a scoring function using all of the following features: price_per_sqm, building_area, how_many_cars, how_many_bathrooms, year_built, distance, type",
    "etc": "I'll leave it to you to figure out what are underrated items and how to create the scoring function.",
}

objective = f"""
The user asked the following question:
{user_input["question"]}

And here are the details:
- purpose: {user_input["purpose"]}
- city: {user_input["city"]}
- rooms: {user_input["rooms"]}
- type: {user_input["type"]}
- budget: {user_input["budget"]}
- top_n: {user_input["top_n"]}
- method: {user_input["method"]}
- etc: {user_input["etc"]}

""".strip()

graph_input = {
    "skip_define_objective_step": False,
    "use_human_in_the_loop": False,
    "variables": [
        Data(
            key="melbourne_housing_df",
            type=DataType.DATAFRAME,
            description="This dataframe contains the melbourne real estate sales data from 2016 to 2017.",
            value=pd.read_csv("./melb_2bed.csv"),
        )
    ],
    "objective": objective,
}

final_chunk = None
for chunk in g.stream(
    graph_input,
    config=config,
    stream_mode="custom",
):
    message = chunk.get("stream_message", "")
    if message:
        print(message)
    else:
        print(chunk)


# Call the graph in while loop
while True:
    try:
        state = g.get_state_history(config)
        lastest_snapshot = next(state)
        message_to_user = lastest_snapshot.interrupts[0].value["message_to_user"]
    except:
        break

    print(f"\n<AGENT>\n{message_to_user}\n</AGENT>")

    user_input = input()

    if user_input.lower() in ["q", "quit"]:
        break
    else:
        for chunk in g.stream(
            Command(resume=user_input.strip()), config=config, stream_mode="custom"
        ):
            message = chunk.get("stream_message", "")
            if message:
                print(message)
            else:
                print(chunk)

state = g.get_state_history(config)
lastest_snapshot = next(state)

with open(f"./results/{current_time}.md", "w") as f:
    f.write(lastest_snapshot.values["final_report"])
    print(f"Final report saved in ./results/{current_time}.md")

1️⃣ Initializing the step 1...
🔍 Reviewing the user request...


Failed to multipart ingest runs: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by ProtocolError('Connection aborted.', TimeoutError('The write operation timed out')))"))
Content-Length: 8096618
API Key: lsv2_********************************************85trace=c5d339c7-8466-4743-98da-c146cf7cd0f7,id=c5d339c7-8466-4743-98da-c146cf7cd0f7; trace=c5d339c7-8466-4743-98da-c146cf7cd0f7,id=def9fde0-2901-4fc4-9b2d-1e89dd59b37d; trace=c5d339c7-8466-4743-98da-c146cf7cd0f7,id=42785ec0-439d-4fb9-8ddd-32ad4f89bc49; trace=c5d339c7-8466-4743-98da-c146cf7cd0f7,id=aa3ec1c0-e722-4deb-9a54-d0e2b6a718e1; trace=c5d339c7-8466-4743-98da-c146cf7cd0f7,id=86ff5d6f-81f8-4f64-ab3a-f6e6a002188b; trace=c5d339c7-8466-4743-98da-c146cf7cd0f7,id=6b5658e1-92a4-45d9-9078-8746b6a

### Example user queries for testing Step 1 define objective.

Sydney real estate is booming. How to become rich by investing in real estate??

Can you find reasonably priced 2-bedroom units for investment in Melbourne?

Can you find underrated top 10 2-bedroom units for investment in Melbourne? My budget is $600k - $900k.

Can you find underrated top 10 2-bedroom units for investment in Melbourne? My budget is $600k - $900k. I'll leave it to you to figure out what are underrated units and how to sort the top 10.

Can you find underrated top 10 2-bedroom units for investment in Melbourne? My budget is $600k - $900k. Use all of the following features for scoring: price_per_sqm, building_area, how_many_cars, how_many_bathrooms, year_built, distance, type
